In [1]:
import os
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
def sigmoid(x) : 
    return 1 / ( 1 + np.exp(-x))
'''
 In ADVI, the variational distribution is taken as gaussian with diagonal variance. p(x) ~ N(mu,std^2)
'''
def variational_pdf(x, mu, std, b, a) : 
    return stats.norm.pdf(x, mu, std) / ((b - a) * sigmoid(x) * (1 - sigmoid(x)))

In [3]:
data1 = np.loadtxt('./data/crystal2.txt',delimiter=',')
data2 = np.loadtxt('./data/crystal3.txt',delimiter=',')
data3 = np.loadtxt('./data/crystal4.txt',delimiter=',')
data4 = np.loadtxt('./data/crystal6.txt',delimiter=',')
data5 = np.loadtxt('./data/crystal7.txt',delimiter=',')

data = np.hstack((data1,data2,data3,data4,data5))

sigma1=np.array([ np.sqrt( (data1[3,i]**2)) for i in range(len(data1[0])) ])
sigma2=np.array([ np.sqrt( (data2[3,i]**2)) for i in range(len(data2[0])) ])
sigma3=np.array([ np.sqrt( (data3[3,i]**2)) for i in range(len(data3[0])) ])
sigma4=np.array([ np.sqrt( (data4[3,i]**2)) for i in range(len(data4[0])) ])
sigma5=np.array([ np.sqrt( (data5[3,i]**2)) for i in range(len(data5[0])) ])

sigma = np.hstack((sigma1,sigma2,sigma3,sigma4,sigma5))

In [4]:
def cosine_func(C,p0,p1,A,w,t0,x):
    return C + p0 * np.exp(-np.log(2) * x / p1) + A * np.cos(w * (x - t0))

In [5]:
with pm.Model() as hyp_1 : 
    C1 = pm.Uniform('C1',lower=0.,upper=400.)
    C2 = pm.Uniform('C2',lower=0.,upper=400.)
    C3 = pm.Uniform('C3',lower=0.,upper=400.)
    C4 = pm.Uniform('C4',lower=0.,upper=400.)
    C5 = pm.Uniform('C5',lower=0.,upper=400.)
    p0_1 = pm.Uniform('p0_1',lower=0.,upper=400.)
    p0_2 = pm.Uniform('p0_2',lower=0.,upper=400.)
    p0_3 = pm.Uniform('p0_3',lower=0.,upper=400.)
    p0_4 = pm.Uniform('p0_4',lower=0.,upper=400.)
    p0_5 = pm.Uniform('p0_5',lower=0.,upper=400.)
    p1_1 = pm.Uniform('p1_1',lower=0.,upper=30000.)
    p1_2 = pm.Uniform('p1_2',lower=0.,upper=30000.)
    p1_3 = pm.Uniform('p1_3',lower=0.,upper=30000.)
    p1_4 = pm.Uniform('p1_4',lower=0.,upper=30000.)
    p1_5 = pm.Uniform('p1_5',lower=0.,upper=30000.)
    A = pm.Uniform('A',lower=0.,upper=400.)
    t0 = pm.Uniform('t0',lower=0.,upper=360.)
    w = pm.Uniform('w',lower=0.0104,upper=0.428)
    mean1 = cosine_func(C1,p0_1,p1_1,A,w,t0,data1[0])
    mean2 = cosine_func(C2,p0_2,p1_2,A,w,t0,data2[0])
    mean3 = cosine_func(C3,p0_3,p1_3,A,w,t0,data3[0])
    mean4 = cosine_func(C4,p0_4,p1_4,A,w,t0,data4[0])
    mean5 = cosine_func(C5,p0_5,p1_5,A,w,t0,data5[0])
    mean = pm.math.concatenate([mean1,mean2,mean3,mean4,mean5])
    pm.Normal('y',mu=mean,sd=sigma,observed=data[1])

In [6]:
with hyp_1:
    advi = pm.ADVI(random_state=1234)
tracker = pm.callbacks.Tracker(
    mean=advi.approx.mean.eval,  # callable that returns mean
    std=advi.approx.std.eval  # callable that returns std
)

In [7]:
approx = advi.fit(1500000,obj_optimizer=pm.adam(learning_rate=5e-5),callbacks=[pm.callbacks.CheckParametersConvergence(diff='absolute'), tracker])

Average Loss = -114.21:  21%|██▏       | 321599/1500000 [01:40<06:06, 3211.94it/s]   
Convergence achieved at 321600
Interrupted at 321,599 [21%]: Average Loss = 2.2035e+08


In [8]:
trace = approx.sample(500)

In [9]:
means_dict = approx.bij.rmap(approx.params[0].eval())
rho_dict = approx.bij.rmap(approx.params[1].eval())
std_dict ={k: np.log(1 + np.exp(v)) for k, v in rho_dict.items()}

# print(means_dict)
# print(std_dict)

In [10]:
def hyp1_logp(trace):
    log_p = 0.
    for var in ['C1','C2','C3','C4','C5','p0_1','p0_2','p0_3','p0_4','p0_5','A']:
        log_p += stats.uniform.logpdf(trace[var],0.,400.)
    for var in ['p1_1','p1_2','p1_3','p1_4','p1_5']:
        log_p += stats.uniform.logpdf(trace[var],0.,30000.)
    log_p += stats.uniform.logpdf(trace['t0'],0.,360.)
    log_p += stats.uniform.logpdf(trace['w'],0.0104,0.428-0.0104)
    
    mean1 = cosine_func(trace['C1'],trace['p0_1'],trace['p1_1'],trace['A'],trace['w'],trace['t0'],data1[0])
    mean2 = cosine_func(trace['C2'],trace['p0_2'],trace['p1_2'],trace['A'],trace['w'],trace['t0'],data2[0])
    mean3 = cosine_func(trace['C3'],trace['p0_3'],trace['p1_3'],trace['A'],trace['w'],trace['t0'],data3[0])
    mean4 = cosine_func(trace['C4'],trace['p0_4'],trace['p1_4'],trace['A'],trace['w'],trace['t0'],data4[0])
    mean5 = cosine_func(trace['C5'],trace['p0_5'],trace['p1_5'],trace['A'],trace['w'],trace['t0'],data5[0])
    mean =  np.hstack([mean1,mean2,mean3,mean4,mean5])
    #log_p += pm.Normal.dist(mu=mean,sd=sigma).logp(data[1]).eval().sum()
    log_p += stats.norm.logpdf(data[1],mean,sigma).sum()
    return log_p

hyp1_logp_vec = np.vectorize(hyp1_logp)

In [11]:
def q(trace,model):
    qw=1.
    l=0.
    u=400.
    for var in model.vars:
        var = str(var).split('~')[0].strip()
        if(str(var)[:len(str(var)) - len("_interval__")]) in ['p1_1','p1_2','p1_3','p1_4','p1_5']:
            l =0.
            u = 30000.
        elif (str(var)[:len(str(var)) - len("_interval__")]) == 'w':
            l = 0.0104
            u = 0.428
        elif (str(var)[:len(str(var)) - len("_interval__")]) == 't0':
            l = 0.
            u = 360.
        else :
            l = 0.
            u = 400.
        qw *= variational_pdf(trace[str(var)],means_dict[str(var)], std_dict[str(var)],u,l)
        #print(qw)
    return qw

q_vec = np.vectorize(q)

In [12]:
# approximate evidence
approx_evidence_1 = (np.exp(hyp1_logp_vec(trace)) / q_vec(trace,hyp_1)).mean()
print('Approx evidence : ', approx_evidence_1)
print('log approx evidence : ', np.log(approx_evidence_1))

Approx evidence :  3.229610974421338e+53
log approx evidence :  123.20937161730973


In [13]:
#ELBO
elbo_1 = (hyp1_logp_vec(trace) - np.log(q_vec(trace,hyp_1))).mean()
print(elbo_1)

114.05411289258159


# Model 2

In [14]:
with pm.Model() as hyp_2 : 
    C1 = pm.Uniform('C1',lower=0.,upper=400.)
    C2 = pm.Uniform('C2',lower=0.,upper=400.)
    C3 = pm.Uniform('C3',lower=0.,upper=400.)
    C4 = pm.Uniform('C4',lower=0.,upper=400.)
    C5 = pm.Uniform('C5',lower=0.,upper=400.)
    p0_1 = pm.Uniform('p0_1',lower=0.,upper=400.)
    p0_2 = pm.Uniform('p0_2',lower=0.,upper=400.)
    p0_3 = pm.Uniform('p0_3',lower=0.,upper=400.)
    p0_4 = pm.Uniform('p0_4',lower=0.,upper=400.)
    p0_5 = pm.Uniform('p0_5',lower=0.,upper=400.)
    p1_1 = pm.Uniform('p1_1',lower=0.,upper=30000.)
    p1_2 = pm.Uniform('p1_2',lower=0.,upper=30000.)
    p1_3 = pm.Uniform('p1_3',lower=0.,upper=30000.)
    p1_4 = pm.Uniform('p1_4',lower=0.,upper=30000.)
    p1_5 = pm.Uniform('p1_5',lower=0.,upper=30000.)
    
    mean1 = cosine_func(C1,p0_1,p1_1,0,0,0,data1[0])
    mean2 = cosine_func(C2,p0_2,p1_2,0,0,0,data2[0])
    mean3 = cosine_func(C3,p0_3,p1_3,0,0,0,data3[0])
    mean4 = cosine_func(C4,p0_4,p1_4,0,0,0,data4[0])
    mean5 = cosine_func(C5,p0_5,p1_5,0,0,0,data5[0])
    mean = pm.math.concatenate([mean1,mean2,mean3,mean4,mean5])
    pm.Normal('y',mu=mean,sd=sigma,observed=data[1])

In [15]:
with hyp_2:
    advi_2 = pm.ADVI(random_state=1234)
tracker_2 = pm.callbacks.Tracker(
    mean=advi_2.approx.mean.eval,  # callable that returns mean
    std=advi_2.approx.std.eval  # callable that returns std
)

In [16]:
approx_2 = advi_2.fit(1500000,obj_optimizer=pm.adam(learning_rate=5e-5),callbacks=[pm.callbacks.CheckParametersConvergence(diff='absolute'), tracker_2])

Average Loss = -126.7:  22%|██▏       | 328599/1500000 [01:28<05:14, 3721.51it/s]    
Convergence achieved at 328600
Interrupted at 328,599 [21%]: Average Loss = 1.8856e+08


In [17]:
trace_2 = approx_2.sample(500)

In [18]:
means_dict = approx_2.bij.rmap(approx_2.params[0].eval())
rho_dict = approx_2.bij.rmap(approx_2.params[1].eval())
std_dict ={k: np.log(1 + np.exp(v)) for k, v in rho_dict.items()}

# print(means_dict)
# print(std_dict)

In [19]:
def hyp2_logp(trace):
    log_p = 0.
    for var in ['C1','C2','C3','C4','C5','p0_1','p0_2','p0_3','p0_4','p0_5']:
        log_p += stats.uniform.logpdf(trace[var],0.,400.)
    for var in ['p1_1','p1_2','p1_3','p1_4','p1_5']:
        log_p += stats.uniform.logpdf(trace[var],0.,30000.)

    
    mean1 = cosine_func(trace['C1'],trace['p0_1'],trace['p1_1'],0,0,0,data1[0])
    mean2 = cosine_func(trace['C2'],trace['p0_2'],trace['p1_2'],0,0,0,data2[0])
    mean3 = cosine_func(trace['C3'],trace['p0_3'],trace['p1_3'],0,0,0,data3[0])
    mean4 = cosine_func(trace['C4'],trace['p0_4'],trace['p1_4'],0,0,0,data4[0])
    mean5 = cosine_func(trace['C5'],trace['p0_5'],trace['p1_5'],0,0,0,data5[0])
    mean =  np.hstack([mean1,mean2,mean3,mean4,mean5])
    log_p += stats.norm.logpdf(data[1],mean,sigma).sum()
    return log_p

hyp2_logp_vec = np.vectorize(hyp2_logp)

In [20]:
# approximate evidence
approx_evidence_2 = (np.exp(hyp2_logp_vec(trace_2)) / q_vec(trace_2,hyp_2)).mean()
print('Approx evidence : ', approx_evidence_2)
print('log approx evidence : ', np.log(approx_evidence_2))

Approx evidence :  9.029215782403812e+57
log approx evidence :  133.44781581851572


In [21]:
#ELBO
elbo_2 = (hyp2_logp_vec(trace_2) - np.log(q_vec(trace_2,hyp_2))).mean()
print(elbo_2)

126.98068307517603


In [22]:
approx_evidence_1/approx_evidence_2

3.576845489411409e-05

In [23]:
bf = (approx_evidence_2/approx_evidence_1)

In [24]:
print(bf)

27957.595679218335


In [25]:
np.log(bf)

10.238444201206004

In [26]:
np.log(bf)/np.log(10)

4.446499819858114

# Nestle

In [6]:
import nestle

In [ ]:
def fit_bg(x,c,p0,p1):
    return c + p0*np.exp(-np.log(2)*x/p1)


def log_likelihood_bg(P):
    
    sigma1=[ data1[3,i] for i in range(len(data1[0])) ]
    y_fit1=fit_bg(data1[0],P[0],P[1],P[2])

    sigma2=[  data2[3,i]  for i in range(len(data2[0])) ]
    y_fit2=fit_bg(data2[0],P[3],P[4],P[5])

    sigma3=[ data3[3,i]  for i in range(len(data3[0])) ]
    y_fit3=fit_bg(data3[0],P[6],P[7],P[8])

    sigma4=[  data4[3,i]  for i in range(len(data4[0])) ]
    y_fit4=fit_bg(data4[0],P[9],P[10],P[11])

    sigma5=[ data5[3,i]  for i in range(len(data5[0])) ]
    y_fit5=fit_bg(data5[0],P[12],P[13],P[14])    

    y_fit=np.hstack((y_fit1,y_fit2,y_fit3,y_fit4,y_fit5))
    sigma=np.hstack((sigma1,sigma2,sigma3,sigma4,sigma5))

    return sum(stats.norm.logpdf(*args) for args in zip(data[1],y_fit,sigma))


a1=100.0*np.max(data[1])
b1=30000.0


def prior_transform_bgnoprior(P):
    return np.array([a1*P[0],a1*P[1],b1*P[2],a1*P[3],a1*P[4],P[5]*b1,a1*P[6],P[7]*a1,P[8]*b1,P[9]*a1,P[10]*a1,P[11]*b1,P[12]*a1,P[13]*a1,P[14]*b1])

result_1 = nestle.sample(log_likelihood_bg, prior_transform_bgnoprior, 15)

print(result_1.logz)     # log evidence
print(result_1.logzerr)

In [ ]:
def fit_cosine(x,c,p0,p1,A,w,t_0):
    return c + p0*np.exp(-np.log(2)*x/p1) + A*np.cos(w*(x-t_0))


def log_likelihood_cosine(P):
    A = P[15]
    w = P[16]
    t_0=P[17]
    sigma1=[ np.sqrt( (data1[3,i]**2)) for i in range(len(data1[0])) ]
    y_fit1=fit_cosine(data1[0],P[0],P[1],P[2],P[15],P[16],P[17])
    
    sigma2=[ np.sqrt( (data2[3,i]**2)) for i in range(len(data2[0])) ]    
    y_fit2=fit_cosine(data2[0],P[3],P[4],P[5],P[15],P[16],P[17])
    
    sigma3=[ np.sqrt( (data3[3,i]**2)) for i in range(len(data3[0])) ]
    y_fit3=fit_cosine(data3[0],P[6],P[7],P[8],P[15],P[16],P[17])
    
    sigma4=[ np.sqrt( (data4[3,i]**2)) for i in range(len(data4[0])) ]
    y_fit4=fit_cosine(data4[0],P[9],P[10],P[11],P[15],P[16],P[17])
    
    sigma5=[ np.sqrt( (data5[3,i]**2) ) for i in range(len(data5[0])) ]
    y_fit5=fit_cosine(data5[0],P[12],P[13],P[14],P[15],P[16],P[17])
    
    sigma=np.hstack((sigma1,sigma2,sigma3,sigma4,sigma5))            
    y_fit=np.hstack((y_fit1,y_fit2,y_fit3,y_fit4,y_fit5))
    
    sigma=np.hstack((sigma1,sigma2,sigma3,sigma4,sigma5))            
    y_fit=np.hstack((y_fit1,y_fit2,y_fit3,y_fit4,y_fit5))
    return sum(stats.norm.logpdf(*args) for args in zip(data[1],y_fit,sigma))


a1=100.0*np.max(data[1])
b1=30000.0

def prior_transform_cos(P):
        return np.array([a1*P[0],a1*P[1],b1*P[2],a1*P[3],a1*P[4],P[5]*b1,a1*P[6],P[7]*a1,P[8]*b1,P[9]*a1,P[10]*a1,P[11]*b1,P[12]*a1,P[13]*a1,P[14]*b1,P[15]*a1,P[16]*0.40759+0.0104,P[17]*361.0])

result_2 = nestle.sample(log_likelihood_cosine, prior_transform_cos, 18)

print(result_2.logz)     # log evidence
print(result_2.logzerr)